In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import os

In [2]:
input_file = 'D:/Calvin/Documents/GTOMSA/Applied Analytics Practicum - CSE 6748/final_data/health_data_final.csv'


def load_dataset(file_path):
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        print(f"Dataset '{file_path}' loaded successfully.\n")
        return df
    else:
        raise FileNotFoundError(f"The file '{file_path}' does not exist. Please check the file path.")

# Load the dataset
df = load_dataset(input_file)


Dataset 'D:/Calvin/Documents/GTOMSA/Applied Analytics Practicum - CSE 6748/final_data/health_data_final.csv' loaded successfully.



In [3]:
# Display the first five rows of the dataset
print("First five rows of the dataset:")
display(df.head())


First five rows of the dataset:


,sustainer,gender,client_state_rank,all_state_rank,client_zip_rank,all_zip_rank,age,account_id,gift_date,gift_min_t12m,...,drtv_t12m,other_t12m,no_channel_t12m,member_t12m,gift_min_lt,gift_max_lt,gift_avg_lt,gift_count_lt,promo_count_lt,cons_month_lt
0,0,0,51,50,899,5504,NaN,8-10344496,2020-09-22,0.0,...,0,1,0,0,0.0,0.0,0.0,1,1,1
1,0,0,51,50,347,2276,NaN,8-11072676,2020-09-22,0.0,...,0,1,0,0,0.0,0.0,0.0,1,1,1
2,0,3,26,35,1060,6340,NaN,8-11096480,2020-09-22,0.0,...,0,1,0,0,0.0,0.0,0.0,1,1,1
3,0,3,43,43,271,1738,NaN,8-11096492,2020-09-22,0.0,...,0,1,0,0,0.0,0.0,0.0,1,1,1
4,0,3,51,50,631,4028,NaN,8-11096533,2020-09-22,0.0,...,0,1,0,0,0.0,0.0,0.0,1,1,1


In [ ]:
# Define the list of columns you want to drop
columns_to_drop = ['all_state_rank','all_zip_rank','tm_t12m','events_t12m','drtv_t12m','no_channel_t12m','member_t12m','age','gift_date','account_id'] 
# all values are single values 

print(f"Columns intended to drop: {columns_to_drop}\n")

existing_columns_to_drop = [col for col in columns_to_drop if col in df.columns]

if existing_columns_to_drop:
    print(f"Columns to be dropped (exist in DataFrame): {existing_columns_to_drop}\n")
else:
    print("No specified columns found in the DataFrame to drop.\n")
df = df.drop(columns=existing_columns_to_drop)
print(f"Successfully dropped columns: {existing_columns_to_drop}\n")

print("Remaining columns after dropping:")
print(df.columns.tolist())


Columns intended to drop: ['all_state_rank', 'all_zip_rank', 'tm_t12m', 'events_t12m', 'drtv_t12m', 'no_channel_t12m', 'member_t12m', 'age', 'gift_date', 'account_id']

Columns to be dropped (exist in DataFrame): ['all_state_rank', 'all_zip_rank', 'tm_t12m', 'events_t12m', 'drtv_t12m', 'no_channel_t12m', 'member_t12m', 'age', 'gift_date', 'account_id']

Successfully dropped columns: ['all_state_rank', 'all_zip_rank', 'tm_t12m', 'events_t12m', 'drtv_t12m', 'no_channel_t12m', 'member_t12m', 'age', 'gift_date', 'account_id']

Remaining columns after dropping:
['sustainer', 'gender', 'client_state_rank', 'client_zip_rank', 'gift_min_t12m', 'gift_max_t12m', 'gift_avg_t12m', 'gift_count_t12m', 'promo_count_t12m', 'cons_month_t12m', 'dm_t12m', 'wm_t12m', 'digital_t12m', 'f2f_t12m', 'other_t12m', 'gift_min_lt', 'gift_max_lt', 'gift_avg_lt', 'gift_count_lt', 'promo_count_lt', 'cons_month_lt']


In [5]:

def handle_missing_values(df, strategy='fill', fill_value=0):
    if strategy == 'fill':
        df_filled = df.fillna(fill_value)
        print(f"Missing values filled with {fill_value}.\n")
        return df_filled
    elif strategy == 'drop':
        df_dropped = df.dropna()
        print("Rows with missing values have been dropped.\n")
        return df_dropped
    else:
        raise ValueError("Unsupported strategy for handling missing values.")

# Handle missing values by filling with -1
df = handle_missing_values(df, strategy='fill', fill_value=-1)

Missing values filled with -1.



In [6]:
#remove_flags = ['dm_t12m','tm_t12m','wm_t12m','digital_t12m','f2f_t12m','events_t12m','drtv_t12m','other_t12m','no_channel_t12m','member_t12m','age','sustainer']
remove_flags = ['dm_t12m','wm_t12m','digital_t12m','f2f_t12m','other_t12m','sustainer']
numerical_columns = df.select_dtypes(include=[np.number]).drop(columns=remove_flags).columns
print("Numerical columns:", numerical_columns.tolist())

Numerical columns: ['gender', 'client_state_rank', 'client_zip_rank', 'gift_min_t12m', 'gift_max_t12m', 'gift_avg_t12m', 'gift_count_t12m', 'promo_count_t12m', 'cons_month_t12m', 'gift_min_lt', 'gift_max_lt', 'gift_avg_lt', 'gift_count_lt', 'promo_count_lt', 'cons_month_lt']


Detecting Outliers We'll detect outliers using two methods:

Z-Score Method, Interquartile Range (IQR) Method

In [7]:
# Function to detect outliers using Z-Score
def detect_outliers_zscore(df, numerical_columns, threshold=3):
    outliers = {}
    print("Detecting outliers using Z-Score method\n")
    for column in numerical_columns:
        if df[column].nunique() > 1:
            z_scores = np.abs(stats.zscore(df[column]))
            outliers_indices = np.where(z_scores > threshold)[0]
            outliers[column] = outliers_indices
            print(f"Z-Score Outliers in '{column}': {len(outliers_indices)} detected")
        else:
            outliers[column] = np.array([])
            print(f"Z-Score Outliers in '{column}': 0 detected (constant column)")
    print("\n")
    return outliers

# Detect outliers using Z-Score
outliers_zscore = detect_outliers_zscore(df, numerical_columns)


Detecting outliers using Z-Score method

Z-Score Outliers in 'gender': 0 detected
Z-Score Outliers in 'client_state_rank': 407 detected
Z-Score Outliers in 'client_zip_rank': 0 detected
Z-Score Outliers in 'gift_min_t12m': 64 detected
Z-Score Outliers in 'gift_max_t12m': 130 detected
Z-Score Outliers in 'gift_avg_t12m': 91 detected
Z-Score Outliers in 'gift_count_t12m': 564 detected
Z-Score Outliers in 'promo_count_t12m': 429 detected
Z-Score Outliers in 'cons_month_t12m': 675 detected
Z-Score Outliers in 'gift_min_lt': 31 detected
Z-Score Outliers in 'gift_max_lt': 89 detected
Z-Score Outliers in 'gift_avg_lt': 104 detected
Z-Score Outliers in 'gift_count_lt': 604 detected
Z-Score Outliers in 'promo_count_lt': 494 detected
Z-Score Outliers in 'cons_month_lt': 673 detected




In [8]:
# Function to detect outliers using IQR
def detect_outliers_iqr(df, numerical_columns):
    outliers = {}
    print("Detecting outliers using IQR method...\n")
    for column in numerical_columns:
        Q1 = df[column].quantile(0.10)
        Q3 = df[column].quantile(0.90)
        IQR = Q3 - Q1
        lower_bound = Q1 - 3 * IQR
        upper_bound = Q3 + 3 * IQR
        condition = (df[column] < lower_bound) | (df[column] > upper_bound)
        outliers_indices = df[condition].index
        outliers[column] = outliers_indices
        print(f"IQR Outliers in '{column}': {len(outliers_indices)} detected")
    print("\n")
    return outliers

# Detect outliers using IQR
outliers_iqr = detect_outliers_iqr(df, numerical_columns)

Detecting outliers using IQR method...

IQR Outliers in 'gender': 0 detected
IQR Outliers in 'client_state_rank': 1057 detected
IQR Outliers in 'client_zip_rank': 0 detected
IQR Outliers in 'gift_min_t12m': 556 detected
IQR Outliers in 'gift_max_t12m': 638 detected
IQR Outliers in 'gift_avg_t12m': 357 detected
IQR Outliers in 'gift_count_t12m': 20 detected
IQR Outliers in 'promo_count_t12m': 0 detected
IQR Outliers in 'cons_month_t12m': 197 detected
IQR Outliers in 'gift_min_lt': 240 detected
IQR Outliers in 'gift_max_lt': 486 detected
IQR Outliers in 'gift_avg_lt': 395 detected
IQR Outliers in 'gift_count_lt': 11 detected
IQR Outliers in 'promo_count_lt': 104 detected
IQR Outliers in 'cons_month_lt': 17 detected




Removing Outliers Based on IQR We'll remove the detected outliers using the IQR method to obtain a dataset without outliers

In [9]:
def remove_outliers_iqr(df, numerical_columns):
    print("Removing outliers based on IQR method...\n")
    Q1 = df[numerical_columns].quantile(0.10)
    Q3 = df[numerical_columns].quantile(0.90)
    IQR = Q3 - Q1
    lower_bound = Q1 - 3 * IQR
    upper_bound = Q3 + 3 * IQR
    condition = ~((df[numerical_columns] < lower_bound) | (df[numerical_columns] > upper_bound)).any(axis=1)
    df_no_outliers = df[condition]
    print("Outliers removed.")
    print(f"Original dataset size: {df.shape[0]} rows")
    print(f"Dataset without outliers size: {df_no_outliers.shape[0]} rows\n")
    return df_no_outliers

# Remove outliers using IQR
df_without_outliers = remove_outliers_iqr(df, numerical_columns)


Removing outliers based on IQR method...

Outliers removed.
Original dataset size: 22581 rows
Dataset without outliers size: 20163 rows



In [10]:

def display_dataset_without_outliers(df):
    print("Displaying the dataset without outliers:\n")
    # Display the first 5 rows
    print("First five rows of the dataset without outliers:")
    display(df.head())
    
    print("\nSummary Statistics of Dataset Without Outliers:")
    display(df.describe())
    
    # Display the number of rows and columns
    print(f"\nDataset without outliers contains {df.shape[0]} rows and {df.shape[1]} columns.\n")

display_dataset_without_outliers(df_without_outliers)


Displaying the dataset without outliers:

First five rows of the dataset without outliers:


,sustainer,gender,client_state_rank,client_zip_rank,gift_min_t12m,gift_max_t12m,gift_avg_t12m,gift_count_t12m,promo_count_t12m,cons_month_t12m,...,wm_t12m,digital_t12m,f2f_t12m,other_t12m,gift_min_lt,gift_max_lt,gift_avg_lt,gift_count_lt,promo_count_lt,cons_month_lt
0,0,0,51,899,0.0,0.0,0.0,1,1,1,...,0,0,0,1,0.0,0.0,0.0,1,1,1
1,0,0,51,347,0.0,0.0,0.0,1,1,1,...,0,0,0,1,0.0,0.0,0.0,1,1,1
4,0,3,51,631,0.0,0.0,0.0,1,1,1,...,0,0,0,1,0.0,0.0,0.0,1,1,1
6,1,3,51,2566,0.0,0.0,0.0,1,1,1,...,0,0,0,1,0.0,0.0,0.0,1,1,1
7,1,1,51,631,0.0,0.0,0.0,1,1,1,...,0,0,0,1,0.0,0.0,0.0,1,1,1



Summary Statistics of Dataset Without Outliers:


,sustainer,gender,client_state_rank,client_zip_rank,gift_min_t12m,gift_max_t12m,gift_avg_t12m,gift_count_t12m,promo_count_t12m,cons_month_t12m,...,wm_t12m,digital_t12m,f2f_t12m,other_t12m,gift_min_lt,gift_max_lt,gift_avg_lt,gift_count_lt,promo_count_lt,cons_month_lt
count,20163.000000,20163.000000,20163.0,20163.000000,20163.000000,20163.000000,20163.000000,20163.000000,20163.000000,20163.000000,...,20163.000000,20163.000000,20163.000000,20163.000000,20163.000000,20163.000000,20163.000000,20163.000000,20163.000000,20163.000000
mean,0.468829,1.376829,51.0,1244.901652,40.284094,64.687668,48.533967,5.289639,2.237316,4.623816,...,0.031692,0.429549,0.001488,0.119575,28.189834,86.982737,45.230775,20.696821,9.090364,6.558796
std,0.499040,1.272294,0.0,777.015334,46.205909,82.956949,54.184359,5.895809,1.968297,8.077100,...,0.175182,0.495024,0.038545,0.324472,33.834579,128.480494,49.366641,28.162194,13.220861,10.089582
min,0.000000,0.000000,51.0,5.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000
25%,0.000000,0.000000,51.0,593.000000,15.000000,20.000000,20.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,10.000000,25.000000,18.750000,2.000000,1.000000,1.000000
50%,0.000000,1.000000,51.0,1165.000000,25.000000,35.000000,27.000000,2.000000,2.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,20.000000,50.000000,27.250000,9.000000,4.000000,2.000000
75%,1.000000,3.000000,51.0,1779.000000,50.000000,75.000000,50.000000,8.000000,3.000000,4.000000,...,0.000000,1.000000,0.000000,0.000000,25.000000,100.000000,50.000000,28.000000,11.000000,7.000000
max,1.000000,3.000000,51.0,3053.000000,360.000000,750.000000,500.000000,49.000000,15.000000,53.000000,...,1.000000,1.000000,1.000000,1.000000,360.000000,1100.000000,500.000000,206.000000,97.000000,82.000000



Dataset without outliers contains 20163 rows and 21 columns.



In [11]:
def save_dataset_without_outliers(df, output_path):
    try:
        df.to_csv(output_path, index=False)
        print(f"Dataset without outliers saved to '{output_path}'.\n")
    except Exception as e:
        print(f"Error saving file: {e}\n")


output_file = 'D:/Calvin/Documents/GTOMSA/Applied Analytics Practicum - CSE 6748/final_data/healthdata_without_outliers.csv'

# Save the cleaned dataset
save_dataset_without_outliers(df_without_outliers, output_file)


Dataset without outliers saved to 'D:/Calvin/Documents/GTOMSA/Applied Analytics Practicum - CSE 6748/final_data/healthdata_without_outliers.csv'.

